In [31]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt

train_images = np.random.randint(256, size=(20, 32, 32, 3))
train_labels = np.random.randint(2, size=(20,1))



In [32]:
#Cuda 혹은 cpu를 사용하시오.
############Write Your Code Here############
device = 'cuda'
############################################


#Custom_Dataset을 정의하시오.(10점)
class Custom_Dataset(Dataset):
    def __init__(self, X, y):
#         데이터셋의 전처리를 해주는 부분
        #입력으로 들어온 X의 pixel값들을 0-1사이로 normalize하고 X의 shape을 (FB,C,H,W)로 변경하여 저장하여 self.X,self.y에 저장하시오.
        self.X = None
        self.y = None
        ############Write Your Code Here############
        self.X = torch.FloatTensor(X)
        self.X = self.X/255
        self.X = self.X.permute(0,3,1,2)
        
        self.y = torch.LongTensor(y)
        
#         print(self.X.shape)
#         print(self.y)
        ############################################
        
    def __len__(self):
#          데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분
        #Custom_Dataset에 저장되어있는 총 data의 개수를 result에 저장하여 반환하시오.
        result = 0
        ############Write Your Code Here############
        self.len = self.y.shape[0]
        ############################################
        return result
    
    def __getitem__(self, idx):
        #self.X, self.y 에서 idx에 맞는 data를 result_X,result_y에 저장하여 반환하시오.
        result_X,result_y = None,None
        ############Write Your Code Here############
        result_X = self.X[idx]
        result_y = self.y[idx]
        ############################################
        return result_X,result_y



In [33]:
    
#torch.nn을 사용하여 아래 함수들을 작성하시오. result는 nn.Layer중 하나이고 result를 반환함.(20점)
def batch_norm(dim,for_MLP=True):
    #for_MLP가 True일 시 MLP를 위한 BN Layer를 반환하고 False일 시 CNN을 위한 BN Layer를 반환함.
    ############Write Your Code Here############
    if for_MLP==True:
        result=nn.BatchNorm1d(dim)
    else:
        result=nn.BatchNorm2d(dim)
    # conv는 4차원(a, b, c, d)
    # mlp는 3차원(a, b, c) 
    # 이기에 다른 놈을 써야한다.
    ############################################
    return result

def fc_layer(in_dim,out_dim):
    #Fully Connected Layer(Dense Layer)
    ############Write Your Code Here############
    result=nn.Linear(in_dim, out_dim)
    ############################################
    return result

def conv_layer(in_ch,out_ch,kernel_size, stride=1, padding=0):
    #Convolutional Layer for image
    ############Write Your Code Here############
    nn.Conv2d(in_channels=in_ch, 
          out_channels=out_ch, 
          kernel_size=kernel_size,
          stride=stride, 
          padding=padding)
    ############################################
    return result

def relu():
    #ReLU function
    ############Write Your Code Here############
    result = nn.ReLU()
    ############################################
    return result

def flatten():
    #Flatten the data
    ############Write Your Code Here############
    result = nn.Flatten()
    ############################################
    return result


#skip_connection(bn -> relu -> conv -> bn -> relu -> conv)를 따르는 Res_block을 만드시오.
#change_res가 True인 res_block을 통과한 feature map은 resolution이 2배 작아지고 channel의 깊이는 2배로 증가함. ex) 32*8*8 -> 64*4*4
#위의 경우에는 skip_connection의 dimension은 1*1 conv로 맞춰줌.
#change_res가 False인 Res_block을 통과한 feature map은 resolution과 channel의 깊이는 그대로 유지됨. ex) 32*4*4 -> 32*4*4(20점)
class Res_block(nn.Module):
    def __init__(self, input_channel, change_res):
        super(Res_block,self).__init__()
        self.change_res = change_res
#         if change_res:
            ############Write Your Code Here############
#             self.conv = conv_layer(input_channel, change_res)
#             self.bn = batch_norm()
            ############################################
#         else:
            ############Write Your Code Here############
            
            ############################################
        ############Write Your Code Here############
            
        ############################################
    def forward(self,X):
        ############Write Your Code Here############
        X = self.batch_norm(X)
        X = self.ReLU(X)
        X = self.conv_layer(X)
        X = self.batch_norm(X)
        X = self.ReLU(X)
        X = self.conv_layer(X)
        
        
        ############################################
        return X

    

In [42]:

#Skip Connection을 이용하여 20개 이상의 layer를 가지고 테스트 셋에대하여 50% 이상의 성능을 주는 MLP를 만드시오.
#nn.ModuleList를 사용하면 많을 층의 layer를 쌓는데 용이함.(20점)
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP,self).__init__()
        ############Write Your Code Here############
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hid_dim = 5
        self.n_layer = 20
        self.act = relu()

        self.fc = nn.Linear(self.input_dim, self.hid_dim)
        self.linears = nn.ModuleList()

        for i in range(self.n_layer-1):
            self.linears.append(nn.Linear(self.hid_dim, self.hid_dim))
    
        self.fc2 = nn.Linear(self.hid_dim, self.out_dim)
        ############################################
    def forward(self,X):
        ############Write Your Code Here############
        X = self.act(self.fc(X))
        for fc in self.linears:
            X = self.act(fc(X))
        X = self.fc2(X)

        ############################################
        return X
        
#Res_Block을 사용하여 테스트 셋에대한 70% 이상의 성능을 주는 CNN 모델을 만드시오.
#flatten전에 nn.AdaptiveAvgPool2d를 사용하면 dimension맞추기가 쉬움.(20점)
class CNN(nn.Module):
    def __init__(self, input_channel, class_number, block_number):
        super(CNN,self).__init__()
        ############Write Your Code Here############
        
        ############################################
    def forward(self,X):
        ############Write Your Code Here############
        
        ############################################
        return X

#loader에 있는 모든 data들에 대한 정확도를 구하여 accuracy에 저장하여 accuracy를 return하는 함수를 구현하시오.(10점)
def evaluate(model, loader):
    model.eval()
    accuracy = 0
    total_example = 0
    correct_example = 0
    for data in loader:
        x,y = data
        x = torch.tensor(x, device = device)
        y = torch.tensor(y, device = device)
        ############Write Your Code Here############
        
        ############################################
    ############Write Your Code Here############
    
    ############################################
    model.train()
    return accuracy

#epoch마다 train_loader에 있는 batch들을 사용하여 모델을 학습하고
#epoch의 마지막 iteration에서는 모델의 validation accuracy를 확인하여 제일 높은 val. acc.를 가진 model을 best_model에 저장하고
#val_acc에는 매 epoch마다 구해진 validation accuracy를 저장하시오.
#running_loss에는 각각의 epoch에서 모든 batch의 loss를 다 더하여 저장하시오.
#모든 epoch의 validation accuracy를 val_acc에 저장하여 best_model과 val_acc를 return하는 함수를 구현하시오.(10점)
def train(model, epoches, train_loader, val_loader, optimizer, criteria):
    best_score = 0
    best_model = None
    batch_len = len(train_loader)
    val_acc = []
    for epoch in range(epoches):
        running_loss = 0
        for i,data in enumerate(train_loader):
            x,y = data
            x = torch.tensor(x, device = device)
            y = torch.tensor(y, device = device)
            ############Write Your Code Here############
            
            ############################################
            
            #epoch의 마지막 iteration
            if i % batch_len == batch_len-1:
                print(f'{epoch+1}th iteration loss :',running_loss/batch_len)
                running_loss = 0
                ############Write Your Code Here############
                
                ############################################
    return best_model, val_acc

In [46]:
#(50점)
#Read the data
trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True)
testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True)

X_train, Y_train = trainset.data, np.array(trainset.targets)
X_test, Y_test = testset.data, np.array(testset.targets)
train_data = Custom_Dataset(trainset.data, trainset.targets)
test_data = Custom_Dataset(testset.data, testset.targets)

#앞서 정의한 Custom_Dataset과 DataLoader를 사용하여 train_loader,val_loader,test_loader를 정의하시오.
#Batch_size는 본인의 컴퓨터 사향에 맞게 변경하면 됨. Validation Set으로 Train Set의 20%를 사용함.
#Preprocessing
train_loader = None
val_loader = None
test_loader = None
batch_size = 128
############Write Your Code Here############
val_size = int(len(train_data.y)/5)
train_size = len(train_data.y) - val_size
indices = list(range(len(train_data.y)))

# torch.utils.data.random_split이 더 나을까?
split = int(np.floor(.2 * len(train_data.y)))
np.random.seed(42)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
train_loader = DataLoader(train_data[train_indices], batch_size=batch_size)
val_loader = DataLoader(train_data[val_indices], batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size)
############################################


#앞서 정의한 MLP,CNN을 사용하여 mlp_model,cnn_model을 정의하시오.
#Define the model
mlp_model = None
cnn_model = None
############Write Your Code Here############
mlp_model = MLP(10, 10)
############################################
'''
mlp_model.to(device)
cnn_model.to(device)


#앞서 정의한 train함수를 사용하여 best_mlp, mpl_val_acc, best_cnn, cnn_val_acc를 구하시오.
#Train the model
best_mlp = None
mlp_val_acc = None
best_cnn = None
cnn_val_acc = None
############Write Your Code Here############

############################################


#앞서 정의한 evaluate함수와 best_model들을 사용하여 mlp_acc, cnn_acc를 구하시오.
#Test Accuracy
mlp_acc = None  
cnn_acc = None 
############Write Your Code Here############

############################################
print('MLP accuracy:',mlp_acc)
print('CNN accuracy:',cnn_acc)


#앞서 구한 val_acc들을 사용하여 이해 가능한 그래프를 그리시오.
#Validation Accuracy Plot
############Write Your Code Here############

############################################
'''

Files already downloaded and verified
Files already downloaded and verified


AttributeError: 'MLP' object has no attribute 'out_dim'

In [36]:
Custom_Dataset(Dataset)

TypeError: __init__() missing 1 required positional argument: 'y'